In [1]:
from __future__ import division # safety with double division
from pyomo.environ import *
from pyomo.opt import SolverFactory

M = AbstractModel()
M.name = "Clustering Linear Program"

## Parameters
- **d**: number of dimensions
- **n**: number of points to cluster
- **c**: number of clusters to generate

In [2]:
M.NumberOfDimensions = Param(within=NonNegativeIntegers)
M.NumberOfPoints = Param(within=NonNegativeIntegers)
M.NumberOfClusters = Param(within=NonNegativeIntegers)

## Set
- **Dimension Index (D)**: Set consisting of all possible possible dimensions an arbitrary point i.e. $[x_1, x_2, x_3 ... x_n]$
- **Points (P)**: Set consisting of all indexes for Points in the system. $[p_1, p_2, p_3 ... p_m]$
- **Cluster Index (C)**: Set consisting of possible ClusterIndex. $[c_1, c_2, c_3 ... c_k]$

In [3]:
M.DimensionIndex = RangeSet(1,M.NumberOfDimensions)
M.PointsIndex = RangeSet(1,M.NumberOfPoints)
M.ClusterIndex = RangeSet(1,M.NumberOfClusters)

## Inputs
- **Point**: $P_{i,j}$ where $i$ $\in$ PointsIndex and $j$ $\in$ DimensionIndex 

In [4]:
M.Point = Param(M.PointsIndex,M.DimensionIndex, within=Reals)

## Variables
- **Centroid**: $C_{i,d}$ where i $\in$ ClusterIndex and d $\in$ dimensionalIndex 
- **Assignment**: $A_{i,j}$ where i $\in$ pointsIndex and j $\in$ clusteringIndex


In [5]:
M.Centroid=Var(M.ClusterIndex, M.DimensionIndex, within=Reals)
M.Assignment=Var(M.PointsIndex, M.ClusterIndex, within=Binary)
M.Slack_Plus = Var(M.PointsIndex,M.ClusterIndex,M.DimensionIndex, within=NonNegativeReals)
M.Slack_Minus = Var(M.PointsIndex,M.ClusterIndex,M.DimensionIndex, within=NonNegativeReals)

## Objective Function
$$ \sum_{i \in Points}\sum_{j \in Clusters}\sum_{x \in Dimensions} A_{i,j}\cdot(S^{+}_{i,j,x}+S^{-}_{i,j,x}) $$

In [6]:
def ObjectiveFunction(M):
    return sum( \
        M.Assignment[i,j]*(M.Slack_Plus[i,j,x]+M.Slack_Minus[i,j,x])\
               for i in M.PointsIndex \
               for j in M.ClusterIndex \
               for x in M.DimensionIndex)
M.Distance = Objective(rule=ObjectiveFunction, sense=minimize)
    

## Constraints

### Constraint 1: Distance Constraint
Used to convert distance metric into 1-norm
$$0=P_{i,x}-C_{j,x}+(S^{+}_{i,j,x}-S^{-}_{i,j,x}) \qquad \forall i \in P, j\in C, x \in d $$

In [7]:
def DistanceConstraint(M,i,j,x):
    return M.C[j,x] == M.P[i,x]+M.Slack_Plus[i,j,x]-M.Slack_Minus[i,j,x]
M.Distance = Constraint(M.PointsIndex, M.ClusterIndex, M.DimensionIndex, rule = DistanceConstraint)

    'pyomo.core.base.objective.SimpleObjective'>) on block Clustering Linear
    Program with a new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
